# trajectory_analyze

particle trajectory analysis

file requirements:
- particle netcdf file generated by parcels after simulation
- the actual ocean data the particles ran on

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.spatial
import xarray as xr

import utils
from constants import *
from parcels_utils import HFRGrid
from parcels_analysis import *

In [ ]:
# the data generated by ParticleFile
p_nc = utils.FILES_ROOT / utils.PARTICLE_NETCDF_DIR / "particle_plume_track.nc"
# particle data
p_ds = xr.open_dataset(p_nc)
p_ds.close()
p_results = ParticleResult(p_ds)

# ocean current netcdf data the particles used
d_nc = utils.FILES_ROOT / utils.CURRENT_NETCDF_DIR / "west_coast_1km_hourly/tj_plume_interped.nc"
# ocean current data
d_info = HFRGrid(d_nc)
d_ds = d_info.xrds
fs = d_info.fieldset

In [ ]:
lats, lons = utils.load_pts_mat(utils.MATLAB_DIR / "coastline.mat", "latz0", "lonz0")
# lats, lons = utils.load_pts_mat(utils.MATLAB_DIR / "coastline_1km.mat", "rlatz0", "rlonz0")
coastline_points = np.array([lats, lons]).T
coast_kdtree = scipy.spatial.KDTree(coastline_points)

tijuana_mouth = np.array([32.551967, -117.127208])

In [ ]:
collide_dist_thresh = 100  # meters
plume_pot_thresh = 500  # meters
feature_coast = ParticlePlotFeature.get_sd_coastline(track_dist=collide_dist_thresh)
feature_station = StationFeature.get_sd_stations(track_dist=plume_pot_thresh)
feature_mouth = LatTrackedPointFeature.get_tijuana_mouth()

In [ ]:
p_results.plot_features = {}
p_results.add_plot_feature(feature_station, name="station")
p_results.add_plot_feature(feature_mouth, name="tijmouth")
frames = p_results.generate_all_plots(utils.FILES_ROOT / utils.PICUTRE_DIR / "stuff")
frames2 = p_results.generate_all_plots(utils.FILES_ROOT / utils.PICUTRE_DIR / "stuff", filename="snapin", feat_info=[], domain=TIJUANA_MOUTH_DOMAIN)

In [ ]:
TIJUANA_MOUTH_DOMAIN = dict(
    S=32.53,
    N=32.564, 
    W=-117.162,
    E=-117.105
)

# domain = TIJUANA_MOUTH_DOMAIN
domain = d_info.get_domain()
domain["W"] = -117.26
domain["S"] = 32.25

# line segment representation of coastline
fig, ax = get_carree_axis(domain)
get_carree_gl(ax)
plt.scatter(feature_coast.lons, feature_coast.lats, s=10)
plt.scatter(feature_station.lons, feature_station.lats, s=30)
# plt.plot(coastline_points.T[1], coastline_points.T[0])